In [ ]:
# Import required libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import requests
import io
import ast

# Import scikit-learn modules for machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Import geospatial data visualization libraries
import geopandas as gpd
import folium
from branca.colormap import linear
from folium import plugins

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Loading the dataset 
url = "https://data.cityofchicago.org/resource/4ijn-s7e5.csv"
params = {"$limit": 1000000} 
response = requests.get(url, params=params)
data = pd.read_csv(io.StringIO(response.text))


In [ ]:
# Display the first few rows of the DataFrame to inspect the data
data.head(5)

In [ ]:
# Check the data types and missing values in the DataFrame
data.info()

In [ ]:
data.isna().sum()

In [ ]:
# Summary statistics of the numerical columns
data.describe()

1. Data Preprocessing

In [ ]:
# column aka_name

In [ ]:
data['aka_name'] = data['aka_name'].fillna(data['dba_name'])

In [ ]:
# column license_

We noticed however that there are some restaurant with licence numbers equal to 0.

handle new license numbers for rows with a license value of 0 or NaN.

In [ ]:
#starting id for new license numbers
new_license_id = int(data['license_'].max() + 1)
license_zero = data[(data['license_'] == 0) | (data['license_'].isna()) ][['dba_name', 'address']].copy().drop_duplicates()
license_zero['new_license'] = [i for i in range(new_license_id, new_license_id+len(license_zero))]
data = data.merge(license_zero, on=['dba_name', 'address'], how='left')
data['license_'] = data['license_'].apply(lambda x: np.nan if x == 0 else x)
data['license_'] = data['license_'].fillna(value = data['new_license'])
data.drop(columns=['new_license'], inplace=True)

In [ ]:
# column facility_type

In [ ]:
data['facility_type'].value_counts().sort_values(ascending=False)

In [ ]:
frequencies_facility_type = data['facility_type'].value_counts(normalize=True)
print(frequencies_facility_type)

In [ ]:
#After analyzing the data, we observe that 'Restaurant' dominates the majority of the records, 
#making up approximately 68% of the dataset, and we use it to fill the missing values.

data['facility_type'].fillna('Unknown Facility', inplace=True)



In [ ]:
# column risk

In [ ]:
# Calculate the frequency of each risk level
risk_counts = data['risk'].value_counts(normalize=True)

# distribution of risk levels
plt.figure(figsize=(8, 6))
plt.bar(risk_counts.index, risk_counts.values)
plt.xlabel('Risk Level')
plt.ylabel('Count')
plt.title('Distribution of Risk Levels in Chicago Food Inspections')
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# The occurrence of the "All" category is relatively small compared to the total dataset and does
# not provide meaningful insights for our analysis. As the number of missing values is relatively small 
# compared to the total data size, and chosen to drop rows with NaN values  and the "All" category from the 'risk' column.

data = data.dropna(subset=['risk'])  
data = data[data['risk'] != 'All']   

In [ ]:
# column address

In [ ]:
# column city

In [ ]:
# Assuming the ZIP codes table is the first table on the page
zip_codes_df = pd.read_csv('zip_code_database.csv', dtype={'zip' :float})
data = pd.merge(data, zip_codes_df[['zip', 'primary_city']], on='zip', how='left')
data['city'] = data['city'].fillna(data['primary_city'])
data.drop('primary_city', axis=1, inplace=True)



In [ ]:
# Fill missing values in 'city' column with 'CHICAGO' where 'state' is 'IL'
data.loc[(data['city'].isna()) & (data['state'] == 'IL'), 'city'] = 'CHICAGO'




In [ ]:
data['city']= data['city'].str.capitalize()

In [ ]:
data['city'].value_counts()

In [ ]:
data[(data['city'].str.contains('icago', case=False)) & (data['city'] != 'Chicago')]['city'].value_counts()


In [ ]:
data.loc[data['city'].str.contains('icago', case=False), 'city'] = 'Chicago'

In [ ]:
# column state

In [ ]:
# Fill missing values in 'state' column with 'IL' where 'city' is 'CHICAGO'
data.loc[(data['state'].isna()) & (data['city'] == 'CHICAGO'), 'state'] = 'IL'

In [ ]:
data['state'].fillna('Not Available', inplace=True)

In [ ]:
data = data[data['state'] == 'IL']

In [ ]:
# Drop the 'state' column
data.drop('state', axis=1, inplace=True)

In [ ]:
# column zip

In [ ]:
data[data['zip'].isna()].city.value_counts()

In [ ]:
data['zip'].value_counts()

In [ ]:
# Find the most frequent zip code
most_frequent_zip = data['zip'].mode().values[0]

# Fill missing 'zip' values with the most frequent zip code
data['zip'].fillna(most_frequent_zip, inplace=True)


# Convert the 'zip' column to string data type 
data['zip'] = data['zip'].astype(int)
data['zip'] = data['zip'].astype(str)

In [ ]:
# column inspection_date

In [ ]:
# Inspection Date to Datetime type
data['inspection_date'] = pd.to_datetime(data['inspection_date'])

In [ ]:
# column inspection_type

In [ ]:
data[data['inspection_type'].isna()]

In [ ]:
data['inspection_type'].value_counts()

In [ ]:
data['inspection_type'].fillna('Unknown', inplace=True)


In [ ]:
data['inspection_type'] = data['inspection_type'].str.lower()
data.loc[data['inspection_type'].str.contains('canvas', case=False), 'inspection_type'] = 'canvass'
data.loc[data['inspection_type'].str.contains('licen', case=False), 'inspection_type'] = 'license'
data.loc[data['inspection_type'].str.contains('complai', case=False), 'inspection_type'] = 'complaint'
data.loc[data['inspection_type'].str.contains('task', case=False), 'inspection_type'] = 'task force'
data.loc[data['inspection_type'].str.contains('kids', case=False), 'inspection_type'] = 'kids cafe'
data.loc[data['inspection_type'].str.contains('out of', case=False), 'inspection_type'] = 'out of business'
data.loc[data['inspection_type'].str.contains('reinspection ', case=False), 'inspection_type'] = 'recent inspection'
# Suspected Food Poisoning replacements
sfp_values = data['inspection_type'].str.lower().str.contains('food|sfp', regex=True)
data.loc[sfp_values, 'inspection_type'] = 'suspected food poisoning'

In [ ]:
def merge_categories(keyword, target_category):
  categories_containing_keyword = data['inspection_type'].str.lower().str.contains(keyword)
  data.loc[categories_containing_keyword, 'inspection_type'] = target_category  

In [ ]:
merge_categories('recent inspection', 'Recent Inspection')
merge_categories('out of business', 'Out of Business')
merge_categories('no entry', 'No Entry')


In [ ]:
data['inspection_type'] = data['inspection_type'].str.title()

In [ ]:
known_types = ['License', 'Canvass', 'Complaint', 'Non-Inspection', 'Suspected Food Poisoning', 'Consultation', 'Tag Removal', 'Recent Inspection', 'Out Of Business', 'Task Force', 'No Entry']
# Classify the rest as unknown
data.loc[~data['inspection_type'].isin(known_types), 'inspection_type'] = 'Unknown'

In [ ]:
# column violations

In [ ]:
# Fill missing 'violations' values with 'Not Available'
data['violations'].fillna('Not Available', inplace=True)

In [ ]:
# Calculate the mean of 'latitude'  
latitude_mean = data['latitude'].mean()

# Fill missing 'latitude'  values with the mean
data['latitude'].fillna(latitude_mean, inplace=True)

In [ ]:
# column longitude

In [ ]:
# Calculate the mean of 'longitude' 
longitude_mean = data['longitude'].mean()

# Fill missing 'longitude' values with the mean
data['longitude'].fillna(longitude_mean, inplace=True)

In [ ]:
# column location

In [ ]:
# recreating the 'location' column by combining 'latitude' and 'longitude'
data['location'] = list(zip(data['latitude'], data['longitude']))

In [ ]:
data.isna().sum()

In [ ]:
data.to_csv('data_prep.csv', index=False)

In [ ]:
# EDA (Exploratory Data Analysis) - Cleaning and Preprocessing

In [ ]:
# Loading the dataset 
data = pd.read_csv('data_prep.csv')

# Display the first few rows of the DataFrame to inspect the data
data.head(5)

In [ ]:
# EDA (Exploratory Data Analysis) - Cleaning and Preprocessing

In [ ]:
# Convert the 'zip' column to string data type 
data['zip'] = data['zip'].astype(int)
data['zip'] = data['zip'].astype(str)

In [ ]:
# for fun
# creating a Word Cloud for the most common words in DBA names

all_dba_names = ' '.join(data['dba_name'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_dba_names)
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
# Extracting Information from 'inspection_date':

# Convert 'inspection_date' column to datetime format
data['inspection_date'] = pd.to_datetime(data['inspection_date'])

# Extract the year from 'inspection_date' and create a new column 'inspection_year'
data['inspection_year'] = data['inspection_date'].dt.year

# Extract the month from 'inspection_date' and create a new column 'inspection_month'
data['inspection_month'] = data['inspection_date'].dt.month

# Determine the season based on the month and create a new column 'inspection_season'
data['inspection_season'] = data['inspection_month'] % 12 // 3 + 1

# Extract the weekday from 'inspection_date' and create a new column 'inspection_weekday'
data['inspection_weekday'] = data['inspection_date'].dt.weekday

In [ ]:
# Data Visualization

# Number of inspections per year
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='inspection_year')
plt.xlabel('Inspection Year')
plt.ylabel('Count')
plt.title('Inspections by Year')
plt.show()



In [ ]:
# Countplot of inspections by year, segmented by 'result'
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='inspection_year', hue=data['results'])
plt.xlabel('Inspection Year')
plt.ylabel('Count')
plt.title('Inspections by Year with Result')
plt.legend(title='Result', loc='upper right')
plt.show()


In [ ]:
# Countplot of inspections by year, segmented by 'risk'
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='inspection_year', hue=data['risk'])
plt.xlabel('Inspection Year')
plt.ylabel('Count')
plt.title('Inspections by Year with Risk')
plt.legend(title='Risk', loc='upper right')
plt.show()

In [ ]:
# Function to extract violation codes from a textual list of violations
def extract_violation_codes(violation):
    # Split the violations and remove leading/trailing spaces
    violations_list = list(map(lambda v: v.strip(), violation.split('|')))
    # Find the dot (.) in each violation to extract the code (e.g., "21. Food Contact Surfaces")
    violation_dots = [violation.find('.') for violation in violations_list]
    # Create a set of unique violation codes to avoid duplicates per inspection
    violation_codes = list(set([int(v[:idx]) for v, idx in zip(violations_list, violation_dots) if idx != -1]))
    return violation_codes

# Add an additional column with extracted violation codes
data['violation Codes'] = data['violations'].apply(extract_violation_codes)


In [ ]:
# Functions for plotting violation codes distribution

In [ ]:
# Function to merge all violation codes from the column into one flat array
def merge_violation_codes(violation_series):
    return [code for inspection_violation_codes in violation_series.values for code in inspection_violation_codes]

# Function to create the histogram for violation codes
def violation_counts(violations, max_violation_code):
    counts, code_bins = np.histogram(violations, bins=np.arange(1, max_violation_code + 2))
    return counts, code_bins

# Function to create the violation codes distribution from the dataframe
def violations_distribution(df, violation_column='Violation Codes', max_violation_code=70):
    all_codes = merge_violation_codes(df[violation_column])
    counts, code_bins = violation_counts(all_codes, max_violation_code)
    return code_bins[:-1], counts



In [ ]:
# Function to plot stacked bars for violation codes distribution
def plot_violations_stacked_bars(data, title, violation_column='violation Codes', max_violation_code=70, xticks=None):
    plt.figure(figsize=(12, 7))
    plt.title(title)

    results = data['results'].unique()
    bars = []
    total_counts = len(data)
    
    previous_counts = np.zeros((max_violation_code,))
    
    # Create stacked bar chart
    for result in results:
        bins, counts = violations_distribution(data[data['results'] == result], violation_column, max_violation_code)
        percentages = counts / total_counts * 100  # Calculate percentage for each violation code count
        bar = plt.bar(bins, percentages, bottom=previous_counts)
        bars.append(bar[0])
        # Accumulate counts for positioning the next stack
        previous_counts += percentages

    if xticks is not None:
        plt.xticks(np.arange(1, max_violation_code + 1), xticks, rotation=40)
    else:
        plt.xlabel('Violation code')

    plt.ylabel('% of inspections with violations')
    plt.ylim((0, 80))

    plt.legend(tuple(bars), tuple(results))
    plt.grid(True, axis='y')

    plt.show()

In [ ]:
plot_violations_stacked_bars(data, 'Distribution of most common violations')


In [ ]:
# Focus only on passed inspections
passed_inspections = data[data['results'] == 'Pass']
plot_violations_stacked_bars(passed_inspections, 'Passed inspection violations distribution')

In [ ]:
# Focus only on failed inspections
failed_inspections = data[data['results'] == 'Fail']
plot_violations_stacked_bars(failed_inspections, 'Failed inspection violations distribution')


In [ ]:
# Based on the above array creates mappings between original codes and generalized categories
def create_mapping(codes):
  mapping = {}
  for new_code, category_codes in enumerate(codes):
      for old_code in category_codes:
          mapping[old_code] = new_code + 1  # new codes starting from 1
  return mapping

# Function to transform array of original codes into our categories
def encode_violations(violations, mapping):
  encoded = []
  for violation in violations:
    encoded.append(mapping[violation])
  return encoded



In [ ]:
food_codes = [11, 12, 13, 14, 15, 17, 23, 26, 27, 28, 30, 31, 37, 39, 42]
facility_codes = [10, 18, 19, 20, 21, 22, 33,34, 35, 36, 38, 41, 43, 44, 48, 50, 51, 53, 55, 56, 59, 60, 62, 64]
sanitary_codes = [2, 8, 16, 40, 45, 46, 47, 49, 52, 54]
staff_codes = [1, 3, 7, 9, 25, 57, 58]
unknown_codes = [4, 5, 6, 24, 29, 32, 61, 63, 70]

codes = [food_codes, facility_codes, sanitary_codes, staff_codes, unknown_codes]

In [ ]:
# Create mappings for conversion
mapping = create_mapping(codes)

In [ ]:
# mapped_inspections_before_change = data['violation Codes'].apply(encode_violations, mapping=before_mapping)
mapped_inspections = data['violation Codes'].apply(encode_violations, mapping=mapping)

In [ ]:
# Add additional column with violation codes by our general categories
data = data.merge(mapped_inspections, left_index=True, right_index=True, suffixes=('', ' Generalized'))

In [ ]:
labels = ['Food', 'Facility conditions', 'Sanitary conditions', 'Staff', 'Other']
plot_violations_stacked_bars(data, '', violation_column='violation Codes Generalized', max_violation_code=5, xticks=labels)

In [ ]:

fig,ax=plt.subplots(2,2,figsize=(15,16))
data.risk.value_counts().plot(kind='bar',color=['red','yellow','green'],ax=ax[0,0])
ax[0,0].tick_params(axis='x',labelrotation=360)
ax[0,0].set_title("The counts of Risk",size=20)
ax[0,0].set_ylabel('counts',size=18)


data.groupby(['inspection_year','risk'])['inspection_id'].agg('count').unstack('risk').plot(ax=ax[0,1],color=['red','yellow','green'])
ax[0,1].legend(loc=0, ncol=1, fontsize=14,bbox_to_anchor=(1.15,0.75))
ax[0,1].set_title("The counts of Risk by year",size=20)
ax[0,1].set_ylabel('counts',size=18)

data.groupby(['inspection_month','risk'])['inspection_id'].agg('count').unstack('risk').plot(ax=ax[1,0],color=['red','yellow','green'])
ax[1,0].legend(loc=0, ncol=1, fontsize=14,bbox_to_anchor=(-0.25,0.75))
ax[1,0].set_title("The counts of Risk by month",size=20)
ax[1,0].set_ylabel('counts',size=18)

sns.scatterplot(x='longitude',y='latitude',hue='risk' ,data=data, ax=ax[1,1])
ax[1,1].set_title("The distribution of inspections by risk",size=20)
ax[1,1].set_xlabel('Longitude')
ax[1,1].set_ylabel('Latitude')


In [ ]:
data_risk_sample=data[data.risk=='Risk 1 (High)'].sample(2500)
Long=data_risk_sample.longitude.mean()
Lat=data_risk_sample.latitude.mean()
risk1_map=folium.Map([Lat,Long],zoom_start=12)

risk1_distribution_map=plugins.MarkerCluster().add_to(risk1_map)
for lat,lon,label in zip(data_risk_sample.latitude,data_risk_sample.longitude,data_risk_sample['dba_name']):
    folium.Marker(location=[lat,lon],icon=None,popup=label).add_to(risk1_distribution_map)
risk1_map.add_child(risk1_distribution_map)

risk1_map

In [ ]:
data_risk_sample=data[data.risk=='Risk 2 (Medium)'].sample(2500)
Long=data_risk_sample.longitude.mean()
Lat=data_risk_sample.latitude.mean()
risk2_map=folium.Map([Lat,Long],zoom_start=12)

risk2_distribution_map=plugins.MarkerCluster().add_to(risk2_map)
for lat,lon,label in zip(data_risk_sample.latitude,data_risk_sample.longitude,data_risk_sample['dba_name']):
    folium.Marker(location=[lat,lon],icon=None,popup=label).add_to(risk2_distribution_map)
risk2_map.add_child(risk1_distribution_map)

risk2_map

In [ ]:
data_risk_sample=data[data.risk=='Risk 3 (Low)'].sample(2500)
Long=data_risk_sample.longitude.mean()
Lat=data_risk_sample.latitude.mean()
risk3_map=folium.Map([Lat,Long],zoom_start=12)

risk3_distribution_map=plugins.MarkerCluster().add_to(risk3_map)
for lat,lon,label in zip(data_risk_sample.latitude,data_risk_sample.longitude,data_risk_sample['dba_name']):
    folium.Marker(location=[lat,lon],icon=None,popup=label).add_to(risk3_distribution_map)
risk3_map.add_child(risk1_distribution_map)

risk3_map

In [ ]:
# Map results to numerical scores
result_scores = {'Pass': 1, 'Fail': 0, 'Pass w/ Conditions': 0.5, 'No Entry': 0.5, 'Not Ready': 0.5, 'Business Not Located': 0,
                'Out of Business': 0}

# Create a new column 'safety_score' with the scores based on 'results'
data['safety_score'] = data['results'].map(result_scores)

# Optional: Round the safety score to 2 decimal places
data['safety_score'] = data['safety_score'].round(2)

In [ ]:
# Group by 'inspection_year' and calculate the mean safety score for each group
grouped_data = data.groupby('inspection_year')['safety_score'].mean().reset_index()

# Bar plot of safety scores by inspection_year type
plt.figure(figsize=(12, 6))
plt.bar(grouped_data['inspection_year'], grouped_data['safety_score'], color='orange')
plt.xlabel('Zip code')
plt.ylabel('Mean Safety Score')
plt.title('Mean Safety Score by Inspection Year')
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
data_safety_sample=data[data['safety_score'] > 0.8].sample(2500)
Long=data_safety_sample.longitude.mean()
Lat=data_safety_sample.latitude.mean()
safety_map=folium.Map([Lat,Long],zoom_start=12)

safety_distribution_map=plugins.MarkerCluster().add_to(safety_map)
for lat,lon,label in zip(data_safety_sample.latitude,data_safety_sample.longitude,data_safety_sample['dba_name']):
    folium.Marker(location=[lat,lon],icon=None,popup=label).add_to(safety_distribution_map)
safety_map.add_child(safety_distribution_map)

safety_map

In [ ]:

# Get the five most frequented facility types
top_facility_types = data['facility_type'].value_counts().nlargest(5).index

# Create a new column 'facility_type_grouped'
data['facility_type_grouped'] = data['facility_type']

# Map facility types not in the top five to 'unknown'
data.loc[~data['facility_type_grouped'].isin(top_facility_types), 'facility_type_grouped'] = 'Unknown Facility'

# Group by 'facility_type' and 'inspection_year', and calculate the mean safety score for each group
grouped_data = data.groupby(['facility_type_grouped', 'inspection_year'])['safety_score'].mean().reset_index()

# Create a pivot table for better visualization
pivot_data = grouped_data.pivot_table(index='facility_type_grouped', columns='inspection_year', values='safety_score')

# Plot grouped bar chart for mean safety scores by facility type for each inspection year
plt.figure(figsize=(12, 6))
pivot_data.plot(kind='bar', cmap='tab20', rot=45, ax=plt.gca())
plt.xlabel('Facility Type')
plt.ylabel('Mean Safety Score')
plt.title('Mean Safety Score by Top Five Facility Types for Each Inspection Year')
plt.legend(title='Inspection Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
# Group by 'facility_type' and 'risk', and calculate the mean safety score for each group
grouped_data = data.groupby(['facility_type_grouped', 'risk'])['safety_score'].mean().reset_index()

# Create a pivot table for better visualization
pivot_data = grouped_data.pivot_table(index='facility_type_grouped', columns='risk', values='safety_score')

# Plot grouped bar chart for mean safety scores by facility type for each risk category
plt.figure(figsize=(12, 6))
pivot_data.plot(kind='bar', cmap='tab20', rot=45, ax=plt.gca())
plt.xlabel('Facility Type')
plt.ylabel('Mean Safety Score')
plt.title('Mean Safety Score by Top Five Facility Types for Each Risk Category')
plt.legend(title='Risk', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Get the top 10 zip codes based on the number of inspections
top_zip_codes = data['zip'].value_counts().nlargest(10).index

# Filter the data to include only the top 10 zip codes
filtered_data = data[data['zip'].isin(top_zip_codes)]

# Group by 'zip' and 'inspection_year', and calculate the mean safety score for each group
grouped_data = filtered_data.groupby(['zip', 'inspection_year'])['safety_score'].mean().reset_index()

# Create a pivot table for better visualization
pivot_data = grouped_data.pivot_table(index='zip', columns='inspection_year', values='safety_score')

# Plot grouped bar chart for mean safety scores by zip code for each inspection year
plt.figure(figsize=(12, 6))
pivot_data.plot(kind='bar', cmap='tab20', rot=45, ax=plt.gca())
plt.xlabel('Zip Code')
plt.ylabel('Mean Safety Score')
plt.title('Mean Safety Score by Top 10 Zip Codes for Each Inspection Year')
plt.legend(title='Inspection Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Group by 'zip' and calculate the mean safety score and facility count based on unique license numbers for each group
grouped_data = data.groupby('zip').agg(safety_score=('safety_score', 'mean'),
                                       facility_count=('license_', 'nunique')).reset_index()

# Convert the 'zip' column to string data type in both DataFrames
grouped_data['zip'] = grouped_data['zip'].astype(int)
grouped_data['zip'] = grouped_data['zip'].astype(str)

# Load the zip code boundaries as a GeoDataFrame
zip_code_boundaries = gpd.read_file('Boundaries - ZIP Codes.geojson')

# Merge the zip code boundaries and grouped_data DataFrames on the 'zip' column
merged_data = zip_code_boundaries.merge(grouped_data, on='zip', how='left')

# Create a map centered on Chicago
map_chicago = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Calculate the min and max safety scores for the color scale
min_score = merged_data['safety_score'].min()
max_score = merged_data['safety_score'].max()

# Create a gradient color scheme based on the safety scores
colormap = linear.YlOrRd_09.scale(min_score, max_score)

# Create a style function to shape the zip code areas on the map with the gradient color scheme
def style_function(feature):
    safety_score = feature['properties']['safety_score']
    return {
        'fillColor': colormap(safety_score),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Create a GeoJSON layer for the zip code boundaries with the style function and tooltip
folium.GeoJson(merged_data, 
               tooltip=folium.GeoJsonTooltip(fields=['zip', 'safety_score', 'facility_count'], 
                                             aliases=['Zip Code', 'Mean Safety Score', 'Facility Count'], 
                                             labels=True),
               style_function=style_function).add_to(map_chicago)

# Add the color scale to the map
colormap.caption = 'Safety Score'
map_chicago.add_child(colormap)

# Add title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Safety Score by Zip Code</b></h3>
             '''
map_chicago.get_root().html.add_child(folium.Element(title_html))

    
    
# Display the map
map_chicago



In [ ]:
# Filter data for specific criteria 
filtered_data = data[
    (data['facility_type'] == 'Restaurant') &
    (data['results'] == 'Pass') &
    (data['zip'].isin(zip_code_boundaries['zip'])) &
    (data['inspection_year'] >= 2023)
]

In [ ]:
# Create a map centered on Chicago
map_chicago = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Add facility locations to the map for the selected zip codes
filtered_facilities = filtered_data.sample(150)
facilities_map = plugins.MarkerCluster().add_to(map_chicago)
for lat, lon, label in zip(filtered_facilities.latitude, filtered_facilities.longitude, filtered_facilities['dba_name']):
    folium.Marker(location=[lat, lon], icon=None, popup=label).add_to(map_chicago)

# Create a GeoJSON layer for the zip code boundaries with the style function and tooltip
folium.GeoJson(merged_data,
               tooltip=folium.GeoJsonTooltip(fields=['zip', 'safety_score', 'facility_count'],
                                             aliases=['Zip Code', 'Mean Safety Score', 'Facility Count'],
                                             labels=True),
               style_function=style_function).add_to(map_chicago)

# Add the color scale to the map
colormap.caption = 'Safety Score'
map_chicago.add_child(colormap)

# Add title to the map
title_html = '''
            <h3 align="center" style="font-size:16px"><b>Safety Score by Zip Code and Restaurants that Passed Inspection in 2023</b></h3>           
            '''
map_chicago.get_root().html.add_child(folium.Element(title_html))

# Display the map
map_chicago



In [ ]:
# Save the processed data to a CSV file
data.to_csv('data_eda.csv', index=False)

In [ ]:
# Loading the dataset 
data = pd.read_csv('data_eda.csv')


In [ ]:
# Display the first few rows of the DataFrame to inspect the data
data.head(5)

In [ ]:
# Convert 'inspection_type' column into dummy variables using one-hot encoding
data = pd.get_dummies(data, columns=['inspection_type'], prefix='', prefix_sep='')

In [ ]:
# Convert 'inspection_type' column into dummy variables using one-hot encoding
data = pd.get_dummies(data, columns=['facility_type_grouped'], prefix='', prefix_sep='')

In [ ]:
# Convert 'inspection_type' column into dummy variables using one-hot encoding
risk_encoded = pd.get_dummies(data['risk'], prefix='risk')
data = pd.concat([data, risk_encoded], axis=1)


In [ ]:
# Create an empty list to store the lengths of violation codes lists
list_lengths = []

# Loop through each row in the DataFrame
for index, row in data.iterrows():
    # Get the list of violation codes for the current row
    violation_codes_list = row['violation Codes']
    
    # Calculate the length of the list and append it to the list_lengths
    list_length = len(violation_codes_list)
    list_lengths.append(list_length)

# Create a new column in the DataFrame to store the list lengths
data['violation_codes_length'] = list_lengths


In [ ]:
# Assuming 'violations' is the name of the column containing the violation codes
# Replace 'violations' with the actual name of your column

food_codes = [11, 12, 13, 14, 15, 17, 23, 26, 27, 28, 30, 31, 37, 39, 42]
facility_codes = [10, 18, 19, 20, 21, 22, 33, 35, 36, 38, 41, 43, 44, 48, 50, 51, 53, 55, 56, 59, 60, 62]
sanitary_codes = [2, 8, 16, 40, 45, 46, 47, 49, 52, 54]
staff_codes = [1, 3, 7, 9, 25, 57, 58]
unknown_codes = [4, 5, 6, 24, 29, 32, 61, 63]

codes_violation = [food_codes, facility_codes, sanitary_codes, staff_codes, unknown_codes]

# Function to safely evaluate and convert the 'violation Codes' string to a list
def safe_eval(code_str):
    try:
        return ast.literal_eval(code_str)
    except (SyntaxError, ValueError):
        return []

# Function to count the violations for each row based on the codes
def count_violations(row):
    # Convert 'violation Codes' string to a list of integers
    violation_codes = safe_eval(row['violation Codes'])
    
    count_per_group = []
    for group_codes in codes_violation:
        count_per_group.append(sum(code in group_codes for code in violation_codes))
    
    return pd.Series(count_per_group, index=['food', 'facility', 'sanitary', 'staff', 'unknown'])

# Apply the count_violations function to the first 10 rows of the DataFrame 'data'
data[['violation_food', 'violation_facility', 'violation_sanitary', 'violation_staff', 'violation_unknown']] = data.apply(count_violations, axis=1)



In [ ]:
# Define a threshold for identifying outliers
violation_threshold = 100

# Identify outlier violation codes
outlier_violations = data['violation Codes'].value_counts()[data['violation Codes'].value_counts() < violation_threshold].index

# Drop rows with outlier violation codes
data = data[~data['violation Codes'].isin(outlier_violations)]


In [ ]:
# Filter the DataFrame to keep only 'Pass' and 'Fail' rows
data = data[data['results'].isin(['Pass', 'Fail'])]

# Map 'Pass' and 'Fail' to 1 and 0, respectively
data['results'] = data['results'].map({'Pass': 1, 'Fail': 0})

data['results'] = data['results'] .astype(int)


In [ ]:
selected_features = ['risk_Risk 1 (High)', 'risk_Risk 2 (Medium)','risk_Risk 3 (Low)',
                    'inspection_year', 'inspection_month',
                    "Children's Services Facility", 'Grocery Store', 'Restaurant', 'School','Unknown Facility',
                    'Canvass', 'Complaint', 'Consultation', 'No Entry', 'Non-Inspection', 'Out Of Business', 'Recent Inspection',
                    'Suspected Food Poisoning', 'Tag Removal', 'Task Force', 'Unknown', 
                    'violation_food', 'violation_facility', 'violation_sanitary', 'violation_staff', 'violation_unknown']

In [ ]:
# Split the data into target variable (safety_score) and features
X = data[selected_features]

y = data['results']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Model 1: Decision Tree Regression
decision_tree_model = DecisionTreeRegressor(max_depth=3, random_state=42)
decision_tree_model.fit(X_train, y_train)

In [ ]:
# Model 2: Random Forest Regression
random_forest_model = RandomForestRegressor(n_estimators=10, random_state=42)
random_forest_model.fit(X_train, y_train)

In [ ]:
# Model 3: Logistic Regression
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

In [ ]:
# Model 4: KNN Regression model
knn_model = KNeighborsRegressor(n_neighbors=7)
knn_model.fit(X_train, y_train)

In [ ]:
# Model 5: Bagging Regression model
bagging_model = BaggingRegressor(n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, make_scorer

# Function to calculate classification metrics
def calculate_classification_metrics(model, X, y_true):
    y_pred = model.predict(X)     
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
   
    return mse,r2

decision_tree_mse, decision_tree_r2 = calculate_classification_metrics(decision_tree_model, X_test, y_test)
random_forest_mse, random_forest_r2 = calculate_classification_metrics(random_forest_model, X_test, y_test)
logistic_regression_mse,logistic_regression_r2 = calculate_classification_metrics(logistic_regression_model, X_test, y_test)
knn_model_mse,  knn_model_r2 = calculate_classification_metrics(knn_model, X_test, y_test)
bagging_model_mse, bagging_model_r2 = calculate_classification_metrics(bagging_model, X_test, y_test)

# Display the evaluation metrics for each model
print("Decision Tree Regression:")
print(f"MSE: {decision_tree_mse:.4f}")
print(f"R2: {decision_tree_r2:.4f}")
print("")

print("Random Forest Regression:")
print(f"MSE: {random_forest_mse:.4f}")
print(f"R2: {random_forest_r2:.4f}")
print("")

print("Logistic Regression:")
print(f"MSE: {logistic_regression_mse:.4f}")
print(f"R2: {logistic_regression_r2:.4f}")
print("")

print("KNN Model:")
print(f"MSE: {knn_model_mse:.4f}")
print(f"R2: {knn_model_r2:.4f}")
print("")

print("Bagging Model:")
print(f"MSE: {bagging_model_mse:.4f}")
print(f"R2: {bagging_model_r2:.4f}")
print("")


Based on the comprehensive evaluation of the models mentioned earlier and their performance metrics, we have selected the Random Forest, Bagging, and KNN Neighbors models for further refinement using GridSearchCV and cross-validation. These models have shown promising results during the initial evaluation, making them ideal candidates for fine-tuning and optimizing their hyperparameters.

In [ ]:
# The hyperparameter grids for each model
random_forest_param_grid = {
    'n_estimators': [10, 15, 20],
    'max_depth': [2, 5, 10],
    'min_samples_split': [2, 5, 10],
}

bagging_param_grid = {
    'n_estimators': [10, 15, 20],  
    'max_samples': [0.5, 0.7, 0.9],  
    'max_features': [0.5, 0.7, 0.9]
}


knn_param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree'],
 }


In [ ]:
# Perform GridSearchCV for each model
random_forest_gridsearch = GridSearchCV(RandomForestRegressor(), random_forest_param_grid, cv=5, scoring='neg_mean_squared_error')
bagging_gridsearch = GridSearchCV(BaggingRegressor(), bagging_param_grid, cv=5, scoring='neg_mean_squared_error')
knn_gridsearch = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the models to the training data
random_forest_gridsearch.fit(X_train, y_train)
bagging_gridsearch.fit(X_train, y_train)
knn_gridsearch.fit(X_train, y_train)

# Perform cross-validation using the best estimator for each model
cv_scores_rf = cross_val_score(random_forest_gridsearch.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_bagging = cross_val_score(bagging_gridsearch.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_knn = cross_val_score(knn_gridsearch.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Print the cross-validation scores for each model
print("Random Forest Cross-Validation Scores:", cv_scores_rf)
print("Bagging Cross-Validation Scores:", cv_scores_bagging)
print("KNN Cross-Validation Scores:", cv_scores_knn)

# Get the best MSE for each model
best_rf_mse = -random_forest_gridsearch.best_score_
best_bagging_mse = -bagging_gridsearch.best_score_
best_knn_mse = -knn_gridsearch.best_score_

# Print the best MSE for each model
print("Best Random Forest MSE:", best_rf_mse)
print("Best Bagging MSE:", best_bagging_mse)
print("Best KNN MSE:", best_knn_mse)

